# Generating Qubit Hamiltonians

In [ ]:
from utility import *
import numpy as np 

Specify the Qubit Hamiltonian of a molecule by its name, internuclear distances, basis set, and fermion-to-qubit transformation.
Here, we show the resulting Hamiltonian for $H_2$ in STO-3G with $0.74\overset{\circ}{A}$ between the $H$ atoms. 

In [2]:
qubit_transf = 'jw' # Jordan-Wigner transformations
h2 = get_qubit_hamiltonian(mol='h2', geometry=0.74, basis='sto3g', qubit_transf=qubit_transf)
print("H:\n{}".format(h2))

H:
-0.09706626816763109 [] +
-0.045302615503799264 [X0 X1 Y2 Y3] +
0.045302615503799264 [X0 Y1 Y2 X3] +
0.045302615503799264 [Y0 X1 X2 Y3] +
-0.045302615503799264 [Y0 Y1 X2 X3] +
0.17141282644776923 [Z0] +
0.16868898170361213 [Z0 Z1] +
0.12062523483390411 [Z0 Z2] +
0.1659278503377034 [Z0 Z3] +
0.17141282644776926 [Z1] +
0.1659278503377034 [Z1 Z2] +
0.12062523483390411 [Z1 Z3] +
-0.2234315369081348 [Z2] +
0.17441287612261586 [Z2 Z3] +
-0.22343153690813478 [Z3]


This Qubit Hamiltonian encodes all of the $2^N$ eigenstates and eigenenergies, where $N=4$ is the number of qubits. 
In principle, one can represent the Hamiltonian as a matrix with $2^N$ dimensions and find the eigenvalues classically through diagonalization, but the cost of this approach clearly grows exponentially with $N$. 

Still, we can check that this approach indeed recovers the ground state energy against method in S1. 

In [4]:
# Defining pauli matrices 
I, X, Y, Z = np.identity(2), np.array([[0, 1], [1, 0]]), np.array([[0, -1j], [1j, 0]]), np.array([[1, 0], [0, -1]])

# Build matrix representiation of the Hamiltonian H
n_qubits = openfermion.count_qubits(h2)
h2_matrix = np.zeros((2**n_qubits, 2**n_qubits), dtype=np.complex)
for term, term_coeff in h2.terms.items(): # Iterate over pauli-words of H
    term = dict(term) # Dict[qubit_index, 'X'/'Y'/'Z']
    
    # Build matrix rep of current pauli-word using kronecker product to represent x_i y_j ...
    pw_matrix = np.identity(1)
    for i in range(n_qubits):
        if i not in term:        pw_matrix = np.kron(pw_matrix, I)
        else:
            if term[i] == 'X':   pw_matrix = np.kron(pw_matrix, X)
            elif term[i] == 'Y': pw_matrix = np.kron(pw_matrix, Y)
            else:                pw_matrix = np.kron(pw_matrix, Z)
    h2_matrix += pw_matrix * term_coeff

eigvals, _ = np.linalg.eigh(h2_matrix)
print("The ground state energy from S1: ")
obtain_PES('h2', [0.74], 'sto-3g', 'fci')
print("\nThe eigenvalues in the matrix representation of Hamiltonian: \n{}".format(eigvals))

The ground state energy from S1: 
converged SCF energy = -1.11675930739643

The eigenvalues in the matrix representation of Hamiltonian: 
[-1.13728383 -0.53820545 -0.53820545 -0.53077336 -0.53077336 -0.53077336
 -0.44561582 -0.44561582 -0.16835243  0.24003549  0.24003549  0.3555207
  0.3555207   0.48314267  0.71510434  0.92317918]


Alternatively, the qubit-tapering technique can find a smaller effective Hamitlonian by subsitituting operators with $\pm 1$. This technique is detailed in Bravyi's work ([Bravyi et al., "Tapering off qubits to simulate fermionic Hamiltonians", arXiv:1701.08213](https://arxiv.org/abs/1701.08213)). 

In [5]:
print("The effective Hamiltonian of h2:\n {}".format(taper_hamiltonian(h2, n_spin_orbitals=4, n_electrons=2, qubit_transf=qubit_transf))) 

The effective Hamiltonian of h2:
 -0.3270705806846181 [] +
0.18121046201519705 [X0] +
-0.789688726711808 [Z0]


We can verify that this new Hamiltonian still encodes the ground state energy. 

In [6]:
# Building the matrix representation of the effective Hamiltonian
h2_matrix = -0.53105134 * I + 0.19679058 * X - 0.53505729 * Z

# Obtain the eigenvalues
eigvals, _ = np.linalg.eigh(h2_matrix)
print("The eigenvalues in the effective Hamiltonian: \n {}".format(eigvals))

The eigenvalues in the effective Hamiltonian: 
 [-1.10115031  0.03904763]


## Step 2: Generating the qubit Hamiltonian
To proceed to VQE one needs to generate the qubit Hamiltonian, the easiest path is via first generating the electronic Hamiltonian in the second quantized form and then transform it into the qubit form using one of the fermion-to-qubit transformation: Jordan-Wigner or Bravyi-Kitaev.  Next, some qubit operators can be substituted by number ($\pm 1$) because their states are stationary for the specific electronic state (e.g., ground state). This reduction is very useful for fitting larger problem in a fewer qubit description and is based on Hamiltonian symmetries.

#### 2.1 What are the requirements for a function of qubit operators to be a valid mapping for the fermionic operators?

First, recall that fermionic creation/annihilation operators in the second quantization formalism are required to obey the anti-commutation relations:
$$ \{  \hat{a}_p, \hat{a}_q^\dagger \} = \delta_{pq} \qquad \{  \hat{a}_p^\dagger, \hat{a}_q^\dagger \} = \{  \hat{a}_p, \hat{a}_q \} = 0 $$

To run our quantum algorithm, we first must transform the Hamiltonian, which is originally expressed in second quantization, as a linear combination of the Pauli operators. In order for the mapping transformation to be valid, we must ensure that it preserves the anti-commutation of the fermionic operators [1]. This is not a given, as Pauli operators do not naturally obey the anti-commutation relations.

To illustrate the transformation, we consider the Jordan-Wigner transformation. Let $|0>_j$ ($|1>_j$) denote the unoccupied (occupied) state of the $j$-th spin orbital. For a single operator, the creation operators could be expressed as:
$$ \hat{a}^\dagger \rightarrow |1><0|_j = \begin{pmatrix} 0 & 0\\ 1 & 0 \end{pmatrix} = \frac{X_j - i Y_j}{2} $$
The problem with this transformation is that the Pauli operators do not satisfy the anti-commutation relation.

There is an easy and intuitive way to change the transformation so that the anti-commutation relation is enforced. We apply Z gates to each qubit from $0$ to $j-1$.
$$ \hat{a}^\dagger \rightarrow \frac{X_j - i Y_j}{2} \otimes Z_0 \otimes \dots \otimes Z_{j-1} $$
One can easily verify that the anti-commutation relation is now enforced.

#### 2.2 The electronic Hamiltonian is real (due to time-reversal symmetry), what consequences does that have on the terms in the qubit Hamiltonian after the Jordan-Wigner transformation?

Time reversal ensures that the fermionic Hamiltonian is real and symmetric. By implication, the Jordan-Wigner transformed qubit Hamiltonian must also be real. If the mapping is performed such that the resulting operators are expressed as products of the Pauli matrices [2] then the assertion that the Hamiltonian be real implies that operators composed of an even number of Y gates are not included. This reduces the set of possible operators, reducing the depth of the circuit.

#### References

[1] Tilly, Jules, et al. "The variational quantum eigensolver: a review of methods and best practices." arXiv preprint arXiv:2111.05176 (2021).

[2] Tang, Ho Lun, et al. "qubit-adapt-vqe: An adaptive algorithm for constructing hardware-efficient ansätze on a quantum processor." PRX Quantum 2.2 (2021): 020310.